## NN using PyTorch
> A typical training procedure for a neural network is as follows:
> 
> Define the neural network that has some learnable parameters (or weights)
> Iterate over a dataset of inputs
> Process input through the network
> Compute the loss (how far is the output from being correct)
> Propagate gradients back into the network’s parameters
> Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MineNN(nn.Module):
    def __init__(self):
        super(MineNN, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = MineNN()
print(net)
        

MineNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
# exploring our model parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [8]:
# random try-out
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0198,  0.0901,  0.0606, -0.0096,  0.1150,  0.0304, -0.0321, -0.0414,
         -0.0066,  0.0713]], grad_fn=<AddmmBackward0>)


In [9]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [10]:
# Loss function
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.9273, grad_fn=<MseLossBackward0>)


In [11]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [12]:
# backpropagation
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0190,  0.0022, -0.0004, -0.0310, -0.0006,  0.0049])
